<a href="https://colab.research.google.com/github/Dadoyen2/RF-classification_ANN_RF_regression-_for-_BBB-permeation/blob/main/RF_regression_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd

# Load the data
file_path = "/content/BBB_features.csv"  # Replace with your file path
BBB_filtered = "BBB_filtered.csv"

# Read the data into a DataFrame
df = pd.read_csv(file_path, sep=";", header=0)

# Filter rows where the name column contains '_c0'
filtered_df = df[df[df.columns[0]].str.contains("_c0", case=False, na=False)]

# Save the filtered DataFrame
filtered_df.to_csv(BBB_filtered, index=False)
print(f"Filtered file saved to: {BBB_filtered}")

# Optional: Download the file in Google Colab
from google.colab import files
files.download(BBB_filtered)


Filtered file saved to: BBB_filtered.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:

file_paths = "/content/BBB_filtered.csv"
BBB_features_new = pd.read_csv(file_paths)
display(BBB_features_new)



,Objects,V,S,R,G,d,W1,W2,W3,W4,...,P3,P4,P5,EMDIF,EMDIS,Sdry/S,Spol/S,FLEX_PT,PAINS,CUSTOM
0,"1,1,1,2-tetrachloroethane_c0",327.875,248.402,1.31994,1.12279,0.511927,560.000,163.875,4.875,0.000,...,0.000000,0.000000,0.000000,3.11199,2.80238,1.000000,0.000000,0.195959,0,NaN
1,"1,1-difluorochloroethylene_c0",210.125,175.335,1.19842,1.03896,0.468668,540.875,335.875,190.000,79.750,...,0.000000,0.000000,0.000000,3.63672,1.35907,0.522885,0.432101,0.000000,0,NaN
2,"1,2-dichloroethane_c0",256.375,205.486,1.24765,1.08036,0.385992,406.875,66.625,0.000,0.000,...,0.000000,0.000000,0.000000,1.50746,2.25570,1.000000,0.000000,0.500000,0,NaN
3,"1,4-divinyloxybutane_c0",430.000,316.145,1.36013,1.22924,0.330687,658.750,302.500,126.625,45.250,...,0.200000,0.000000,0.000000,2.50582,2.93298,0.683736,0.135974,5.533990,0,NaN
4,16a-fluoroestradiol_c0,632.125,417.592,1.51374,1.26940,0.459359,1104.750,704.750,315.625,119.500,...,0.086957,0.086957,0.000000,5.96007,1.36324,0.583919,0.271856,0.000000,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,zanapezil_c0,931.375,614.992,1.51445,1.53976,0.404278,1331.750,607.375,238.500,92.375,...,0.035714,0.035714,0.035714,7.53167,5.76972,0.854906,0.142695,2.319440,0,NaN
326,zidovudine_c0,553.125,389.404,1.42044,1.30632,0.483148,1155.630,914.750,569.375,269.000,...,0.200000,0.200000,0.000000,7.86730,3.57688,0.346269,0.653731,0.848880,1,NaN
327,ZINC04226975_c0,494.125,354.873,1.39240,1.27218,0.413440,900.625,540.500,312.625,119.750,...,0.066667,0.133333,0.000000,5.95301,3.08573,0.683714,0.316286,0.915790,0,NaN
328,zolantidine_c0,911.250,626.518,1.45447,1.61822,0.418693,1421.000,706.625,330.750,80.375,...,0.074074,0.037037,0.037037,7.39785,11.46210,0.766650,0.121874,4.248470,0,NaN


In [55]:
print(BBB_features_new.columns)


Index(['Objects', 'V', 'S', 'R', 'G', 'd', 'W1', 'W2', 'W3', 'W4',
       ...
       'P3', 'P4', 'P5', 'EMDIF', 'EMDIS', 'Sdry/S', 'Spol/S', 'FLEX_PT',
       'PAINS', 'CUSTOM'],
      dtype='object', length=128)


In [63]:
import pandas as pd

# Step 1: Load the BBB.labels file
labels_file = "/content/BBB_labels.txt"
labels_data = []

# Read labels and targets into a list of dictionaries
with open(labels_file, "r") as file:
    for line in file:
        parts = line.strip().split()
        name = parts[0]  # Name of the chemical
        value = float(parts[-1])  # Target value
        labels_data.append({"Name": name, "Target": value})

# Convert to a DataFrame
df_labels = pd.DataFrame(labels_data)

#display(df_labels)



# Step 3: Clean the names in BBB_features to match BBB_labels
BBB_features_new['Cleaned_Name'] = BBB_features_new['Objects'].str.replace('_c0', '', regex=False)

# Step 1: Replace the 'Objects' column with 'Cleaned_Name' values
BBB_features_new['Objects'] = BBB_features_new['Cleaned_Name']

# Step 2: Drop the 'Cleaned_Name' column
BBB_features_new = BBB_features_new.drop(columns=['Cleaned_Name'])

# Display the updated DataFrame




#display(BBB_features_new)


# Step 4: Merge the data to align targets using Cleaned_Name
df_merged = BBB_features_new.merge(df_labels, left_on="Objects", right_on="Name", how="left")

display(df_merged)




# Step 2: Find rows where the Target column is NaN (unmatched names)
unmatched_names = df_merged[df_merged['Target'].isna()]['Objects']

# Step 3: Print the unmatched names
print("Names in features not found in labels:")
print(unmatched_names.to_list())

# Optional: Display the count of unmatched names
print(f"Total unmatched names: {len(unmatched_names)}")


,Objects,V,S,R,G,d,W1,W2,W3,W4,...,P5,EMDIF,EMDIS,Sdry/S,Spol/S,FLEX_PT,PAINS,CUSTOM,Name,Target
0,"1,1,1,2-tetrachloroethane",327.875,248.402,1.31994,1.12279,0.511927,560.000,163.875,4.875,0.000,...,0.000000,3.11199,2.80238,1.000000,0.000000,0.195959,0,NaN,"1,1,1,2-tetrachloroethane",0.33
1,"1,1-difluorochloroethylene",210.125,175.335,1.19842,1.03896,0.468668,540.875,335.875,190.000,79.750,...,0.000000,3.63672,1.35907,0.522885,0.432101,0.000000,0,NaN,"1,1-difluorochloroethylene",-0.02
2,"1,2-dichloroethane",256.375,205.486,1.24765,1.08036,0.385992,406.875,66.625,0.000,0.000,...,0.000000,1.50746,2.25570,1.000000,0.000000,0.500000,0,NaN,"1,2-dichloroethane",-0.14
3,"1,4-divinyloxybutane",430.000,316.145,1.36013,1.22924,0.330687,658.750,302.500,126.625,45.250,...,0.000000,2.50582,2.93298,0.683736,0.135974,5.533990,0,NaN,"1,4-divinyloxybutane",0.12
4,16a-fluoroestradiol,632.125,417.592,1.51374,1.26940,0.459359,1104.750,704.750,315.625,119.500,...,0.000000,5.96007,1.36324,0.583919,0.271856,0.000000,0,NaN,16a-fluoroestradiol,-0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,zanapezil,931.375,614.992,1.51445,1.53976,0.404278,1331.750,607.375,238.500,92.375,...,0.035714,7.53167,5.76972,0.854906,0.142695,2.319440,0,NaN,zanapezil,1.14
326,zidovudine,553.125,389.404,1.42044,1.30632,0.483148,1155.630,914.750,569.375,269.000,...,0.000000,7.86730,3.57688,0.346269,0.653731,0.848880,1,NaN,zidovudine,-0.74
327,ZINC04226975,494.125,354.873,1.39240,1.27218,0.413440,900.625,540.500,312.625,119.750,...,0.000000,5.95301,3.08573,0.683714,0.316286,0.915790,0,NaN,ZINC04226975,-0.87
328,zolantidine,911.250,626.518,1.45447,1.61822,0.418693,1421.000,706.625,330.750,80.375,...,0.037037,7.39785,11.46210,0.766650,0.121874,4.248470,0,NaN,zolantidine,0.14


Names in features not found in labels:
[]
Total unmatched names: 0


In [69]:


#df_merged.drop(columns=["Objects", "Name"], inplace=True)
#df_merged.drop(columns=["CUSTOM"], inplace=True)
display(df_merged)

# Step 6: Save the updated file
new_file = "BBB_features_with_targets.csv"
df_merged.to_csv(new_file, index=False)

print(f"Updated file saved to: {new_file}")

,V,S,R,G,d,W1,W2,W3,W4,W5,...,P3,P4,P5,EMDIF,EMDIS,Sdry/S,Spol/S,FLEX_PT,PAINS,Target
0,327.875,248.402,1.31994,1.12279,0.511927,560.000,163.875,4.875,0.000,0.000,...,0.000000,0.000000,0.000000,3.11199,2.80238,1.000000,0.000000,0.195959,0,0.33
1,210.125,175.335,1.19842,1.03896,0.468668,540.875,335.875,190.000,79.750,16.000,...,0.000000,0.000000,0.000000,3.63672,1.35907,0.522885,0.432101,0.000000,0,-0.02
2,256.375,205.486,1.24765,1.08036,0.385992,406.875,66.625,0.000,0.000,0.000,...,0.000000,0.000000,0.000000,1.50746,2.25570,1.000000,0.000000,0.500000,0,-0.14
3,430.000,316.145,1.36013,1.22924,0.330687,658.750,302.500,126.625,45.250,15.125,...,0.200000,0.000000,0.000000,2.50582,2.93298,0.683736,0.135974,5.533990,0,0.12
4,632.125,417.592,1.51374,1.26940,0.459359,1104.750,704.750,315.625,119.500,52.250,...,0.086957,0.086957,0.000000,5.96007,1.36324,0.583919,0.271856,0.000000,0,-0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,931.375,614.992,1.51445,1.53976,0.404278,1331.750,607.375,238.500,92.375,45.375,...,0.035714,0.035714,0.035714,7.53167,5.76972,0.854906,0.142695,2.319440,0,1.14
326,553.125,389.404,1.42044,1.30632,0.483148,1155.630,914.750,569.375,269.000,123.500,...,0.200000,0.200000,0.000000,7.86730,3.57688,0.346269,0.653731,0.848880,1,-0.74
327,494.125,354.873,1.39240,1.27218,0.413440,900.625,540.500,312.625,119.750,61.375,...,0.066667,0.133333,0.000000,5.95301,3.08573,0.683714,0.316286,0.915790,0,-0.87
328,911.250,626.518,1.45447,1.61822,0.418693,1421.000,706.625,330.750,80.375,30.250,...,0.074074,0.037037,0.037037,7.39785,11.46210,0.766650,0.121874,4.248470,0,0.14


Updated file saved to: BBB_features_with_targets.csv


In [45]:
#BBB_features_new.drop(columns=['CUSTOM'], inplace=True)
#BBB_features_new.drop(columns= ['Objects'], inplace=True)


In [44]:
#display(BBB_features_new)


,V,S,R,G,d,W1,W2,W3,W4,W5,...,P2,P3,P4,P5,EMDIF,EMDIS,Sdry/S,Spol/S,FLEX_PT,PAINS
0,327.875,248.402,1.31994,1.12279,0.511927,560.000,163.875,4.875,0.000,0.000,...,0.0,0.000000,0.000000,0.000000,3.11199,2.80238,1.000000,0.000000,0.195959,0
1,210.125,175.335,1.19842,1.03896,0.468668,540.875,335.875,190.000,79.750,16.000,...,0.0,0.000000,0.000000,0.000000,3.63672,1.35907,0.522885,0.432101,0.000000,0
2,256.375,205.486,1.24765,1.08036,0.385992,406.875,66.625,0.000,0.000,0.000,...,0.0,0.000000,0.000000,0.000000,1.50746,2.25570,1.000000,0.000000,0.500000,0
3,430.000,316.145,1.36013,1.22924,0.330687,658.750,302.500,126.625,45.250,15.125,...,0.0,0.200000,0.000000,0.000000,2.50582,2.93298,0.683736,0.135974,5.533990,0
4,632.125,417.592,1.51374,1.26940,0.459359,1104.750,704.750,315.625,119.500,52.250,...,0.0,0.086957,0.086957,0.000000,5.96007,1.36324,0.583919,0.271856,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,931.375,614.992,1.51445,1.53976,0.404278,1331.750,607.375,238.500,92.375,45.375,...,0.0,0.035714,0.035714,0.035714,7.53167,5.76972,0.854906,0.142695,2.319440,0
326,553.125,389.404,1.42044,1.30632,0.483148,1155.630,914.750,569.375,269.000,123.500,...,0.0,0.200000,0.200000,0.000000,7.86730,3.57688,0.346269,0.653731,0.848880,1
327,494.125,354.873,1.39240,1.27218,0.413440,900.625,540.500,312.625,119.750,61.375,...,0.0,0.066667,0.133333,0.000000,5.95301,3.08573,0.683714,0.316286,0.915790,0
328,911.250,626.518,1.45447,1.61822,0.418693,1421.000,706.625,330.750,80.375,30.250,...,0.0,0.074074,0.037037,0.037037,7.39785,11.46210,0.766650,0.121874,4.248470,0


In [47]:

"""
# Step 1: Load BBB.labels file and extract target values
labels_file = "/content/BBB_labels.txt"
targets = []

# Read the text file and extract numeric values
with open(labels_file, "r") as file:
    for line in file:
        value = float(line.strip().split()[-1])  # Extract the last column (numeric value)
        targets.append(value)

# Step 2: Load BBB.features file
#features_file = "/content/BBB.features.csv"  # Adjust path if needed
#df_features = pd.read_csv(features_file)

# Step 3: Check consistency of lengths
#if len(BBB_features_new) != len(targets):
 #   raise ValueError("The number of rows in BBB_features_new and target values extracted do not match!")

# Step 4: Add target values as a new column
BBB_features_new['target'] = targets
display(BBB_features_new)

# Step 5: Save the updated file
#output_file = "/content/BBB_updated_features.csv"
#df_features.to_csv(output_file, index=False)

#print(f"Updated file saved successfully to: {output_file}")
"""

ValueError: Length of values (331) does not match length of index (330)